In [3]:
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
from ipywidgets import VBox, HBox, Output, Button, IntText, Label, FloatText

import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from matplotlib.collections import PathCollection
import matplotlib.animation as animation

/Users/nikolashahpazov/.pyenv/versions/3.9.2/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


# The Metropolis-Hastings Algorithm

# Suppose we want to sample from a whacky distrbution such as:
$$
P(x) = \frac{exp(-x^2)(2 + sin(5x) + sin(2x) )}{\int_{-\infty}^{\infty} exp(-x'^2)(2 + sin(5x') + sin(2x') )  dx^{'}}
$$

In [4]:
def targetdist(x):
        # the numerator that we care about
        probX = np.exp(-x**2) * (2 + np.sin(x*5) + np.sin(x*2))
        return probX
x = np.arange(-3,3, 0.01)
y = targetdist(x)

px.line(x = x, y= y)

## Might be tricky or impossible to solve that integral, so I know the distribution up to some constant:
$ \Large  P(x)    \propto exp(-x^2)(2 + sin(5x) + sin(2x) ) $

## How can I generate samples from this distribution?

### Define a Markov chain over possible x values, so that the stationary distribution of the Markov Chain is the $P(X)$. Use Markov chain to generate a sequence of $x$ values, denoted $(x_{0},x_{1},x...)$, so that as $n \to \infty$, we can guarantee that $x_{n} \sim P(x)$ 

# How do we go about going this?

### We know the current state $x_{n}$, and we want to generate $x_{n+1}$. This is a 2 step process.
### 1st step is to generate a candidate $x^{*}$, which is generated from a proposal distribution $Q(X^{*} | x_{n})$ which depends on the current state of the Markov chain $x_{n}$. We need a distribution that is centred on $x_n$, so can use something like:

### $x^{*} | x_{n} \sim Normal(x_{n}, \sigma^{2})$ where we punch in our own $\sigma$.

### 2nd step is the accept-reject step. We calculate an acceptance probability $A(x_{n} \to x^{*})$ which is given by:

$ \Large A(x_{n} \to x^{*})   = min(1, \frac{P(x^{*})}{P(x_{n})} x \frac{Q(x_{n} | x^{*} )}{Q(x^{*} | x_{n})} )$ 

### $\frac{P(x^{*})}{P(x_{n})}$ is easy to compute, as we just plug the value of $x^{*}$ and $x_{n}$ in the numerator above.
### Looking at, $\frac{Q(x_{n} | x^{*} )}{Q(x^{*} | x_{n})}$  $Q(x^{*} | x_{n})$ tells you the probability of generating  $x^{*}$ as the candidate given the current state $ x_{n})$, which is what is happening now. The numerator tells us exactly the opposite. If the distribution is symmetric, then these 2 numbers will be the same and it will just become 1. This is the Metropolis algo, and it's a special case of the Metropolis-Hastings algo.


### We now have a candidate $x^{*}$ and calculated the acceptance probability $A(x_{n} \to x^{*}$, we have to either accept or reject this value. If we reject then $x_{n+1} = x_{n}$. This is easy. We just generate a random number from a uniformly distributed distribution between 0 and 1 which we call $u$ :

$$ x_{n+1}  =  \begin{equation}
 \left\{ 
  \begin{aligned}
 x^{*} \quad \text{if} \quad u \leq A(x_{n} \to x^{*}) \\
 x_{n} \quad \text{if} \quad u > A(x_{n} \to x^{*}) \\
  \end{aligned}
  \right.
\end{equation}
$$

In [16]:
def run_mcmc(caller):
    out.clear_output()
    display_label.value = 'Running MCMC'
        ### parameters ###
    burnin = burin_input.value  # number of burn-in iterations
    lag = lag_input.value  # iterations between successive samples
    nsamp = nsamp_input.value # number of samples to draw
    sig = sig_input.value # standard deviation of Gaussian proposal
    x = start_point_input.value # start point
    ### storage ###
    X = np.zeros((nsamp, 1)) # samples drawn from the Markov chain
    acc = np.array((0, 0))  # vector to track the acceptance rate


    def MHstep(x0, sig):
        xp = np.random.normal(loc = x0, scale = sig)  # generate candidate from Gaussian
        accprob = targetdist(xp) / targetdist(x0) # acceptance probability
        u = np.random.rand() # uniform random number
        if u <= accprob: # if accepted
            x1 = xp # new point is the candidate
            a = 1 # note the acceptance
        else: # if rejected
            x1 = x0 # new point is the same as the old one
            a = 0 # note the rejection
        return x1, a

    def targetdist(x):
        return np.exp(-x**2) * (2 + np.sin(x*5) + np.sin(x*2))


    # MH routine
    for i in range(burnin):
        x,a = MHstep(x, sig); # iterate chain one time step
        acc = acc + np.array((a, 1)) # track accept-reject status

    for i in range(nsamp):
        for j in range(lag):
            x, a = MHstep(x, sig) # iterate chain one time step
            acc = acc + np.array((a, 1)) # track accept-reject status
        X[i] = x # store the i-th sample
    df = pd.DataFrame(data=X, columns = ['Trace'])
    
    display_label.value = 'Average Acceptance: ' + str(round(acc[0] / acc[1],2))
    
    
    with out:
        fig, axs = plt.subplots(2, 1)

        axs[0].hist(df.values, bins=20)
        axs[1].plot(df)

        r = Affine2D().rotate_deg(90)

        fig = plt.gcf()
        fig.set_size_inches(8, 6)


        for x in axs[1].images + axs[1].lines + axs[1].collections:
            trans = x.get_transform()
            x.set_transform(r+trans)
            if isinstance(x, PathCollection):
                transoff = x.get_offset_transform()
                x._transOffset = r+transoff

        old = axs[1].axis()
        axs[1].axis(old[2:4] + old[0:2])

        plt.show()

In [17]:
%matplotlib inline
run_button = Button(description = 'Run MCMC')
burin_input = IntText(value=0, description = 'Burn-in')
lag_input = IntText(value=1, description ='Lag')
nsamp_input = IntText(value=1000, description ='Num Samples')
sig_input = FloatText(value=1.0, description ='Sigma')
start_point_input = FloatText(value='-1', description ='Start Value')
display_label  = Label(value = 'Ready to go!')
    
out = Output()

run_button.on_click(run_mcmc)

all_widgets = HBox([VBox([HBox([run_button, display_label]) ,
                    burin_input,
                    lag_input,
                    nsamp_input,
                    sig_input,
                    start_point_input]),
                    out])

display(all_widgets)

In [15]:
### parameters ###
burnin = 10  # number of burn-in iterations
lag = 1  # iterations between successive samples
nsamp = 1000 # number of samples to draw
sig = 1 # standard deviation of Gaussian proposal
x = -1 # start point
### storage ###
X = np.zeros((nsamp,1)) # samples drawn from the Markov chain
acc = np.array((0, 0))  # vector to track the acceptance rate


def MHstep(x0,sig):
    xp = np.random.normal(loc = x0, scale = sig)  # generate candidate from Gaussian
    accprob = targetdist(xp) / targetdist(x0) # acceptance probability
    u = np.random.rand() # uniform random number
    if u <= accprob: # if accepted
        x1 = xp # new point is the candidate
        a = 1 # note the acceptance
    else: # if rejected
        x1 = x0 # new point is the same as the old one
        a = 0 # note the rejection
    return x1, a

def targetdist(x):
    probX = np.exp(-x**2) * (2 + np.sin(x*5) + np.sin(x*2))
    return probX


# MH routine
for i in range(burnin):
    x,a = MHstep(x,sig); # iterate chain one time step
    acc = acc + np.array((a, 1)) # track accept-reject status

for i in range(nsamp):
    for j in range(lag):
        x,a = MHstep(x,sig) # iterate chain one time step
        acc = acc + np.array((a, 1)) # track accept-reject status
    X[i] = x # store the i-th sample

In [9]:
acc[0] / acc[1]

0.48415841584158414

In [14]:
df = pd.DataFrame(data=X, columns = ['Trace'])

In [13]:
fig = ff.create_distplot([X.reshape(-1)], group_labels=['plot'], bin_size=0.1, show_rug=False)
fig.show()

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from matplotlib.collections import PathCollection
import matplotlib.animation as animation
%matplotlib widget

fig, axs = plt.subplots(2, 1)

axs[0].hist(df.values, bins=20)
axs[1].plot(df)


r = Affine2D().rotate_deg(90)

fig = plt.gcf()
fig.set_size_inches(8, 6)


def animate(i):
    data =  df.iloc[:int(i+1)]
    axs[0].clear()
    axs[1].clear()
    
    axs[0].hist(data.values, bins=20)
    axs[1].plot(data)
    
    for x in axs[1].images + axs[1].lines + axs[1].collections:
        trans = x.get_transform()
        x.set_transform(r+trans)
        if isinstance(x, PathCollection):
            transoff = x.get_offset_transform()
            x._transOffset = r+transoff
        
    old = axs[1].axis()
    axs[1].axis(old[2:4] + old[0:2])

ani = animation.FuncAnimation(fig, func=animate, frames=len(df), interval=1, repeat =False)
plt.show()

ModuleNotFoundError: No module named 'ipympl'